# Cargar datos

Load drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


load datasets

In [ ]:
import pandas as pd


x_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset_bitbot/x_train.csv')
y_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset_bitbot/y_train.csv')
x_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset_bitbot/x_test.csv')
y_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset_bitbot/y_test.csv')

# XGBoost

liberias para importar xgboost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, matthews_corrcoef, classification_report
from sklearn.model_selection import RandomizedSearchCV


Ajuste de columnas necesarias

In [ ]:
# Lista de columnas problemáticas
cols_time = ['fecha', 'hora_inicial', 'hora_final']

# Convertir a datetime
for col in cols_time:
    x_train[col] = pd.to_datetime(x_train[col], errors='coerce')
    x_test[col] = pd.to_datetime(x_test[col], errors='coerce')

# Convertir datetime -> timestamp en segundos (int64)
for col in cols_time:
    x_train[col] = x_train[col].astype('int64') // 10**9
    x_test[col] = x_test[col].astype('int64') // 10**9


x_train = x_train.drop('date', axis=1)
x_test = x_test.drop('date', axis=1)

/tmp/ipython-input-1718941226.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  x_train[col] = pd.to_datetime(x_train[col], errors='coerce')
/tmp/ipython-input-1718941226.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  x_test[col] = pd.to_datetime(x_test[col], errors='coerce')
/tmp/ipython-input-1718941226.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  x_train[col] = pd.to_datetime(x_train[col], errors='coerce')
/tmp/ipython-input-1718941226.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensu

In [ ]:
Observacion de las columnas


In [ ]:
print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527192 entries, 0 to 527191
Data columns (total 32 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   entrada_vela_1             527192 non-null  float64
 1   salida_vela_1              527192 non-null  float64
 2   Max_vela_1                 527192 non-null  float64
 3   Min_vela_1                 527192 non-null  float64
 4   porcentaje_retorno_vela_1  527192 non-null  float64
 5   entrada_vela_2             527192 non-null  float64
 6   salida_vela_2              527192 non-null  float64
 7   Max_vela_2                 527192 non-null  float64
 8   Min_vela_2                 527192 non-null  float64
 9   porcentaje_retorno_vela_2  527192 non-null  float64
 10  entrada_vela_3             527192 non-null  float64
 11  salida_vela_3              527192 non-null  float64
 12  Max_vela_3                 527192 non-null  float64
 13  Min_vela_3                 52

Implementacion del modelo base

In [ ]:
model_base = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42
)

model_base.fit(x_train, y_train)

# Evaluación del modelo base
y_pred_base = model_base.predict(x_test)
print("Accuracy base:", accuracy_score(y_test, y_pred_base))

Accuracy base: 0.5595304936341978


Ajuste de hiperparámetros (Grid Search)

In [ ]:
param_dist = {
    "n_estimators": np.arange(200, 800, 100),
    "max_depth": np.arange(3, 10),
    "learning_rate": np.logspace(-3, -1, 10),
    "subsample": np.linspace(0.5, 1.0, 6),
    "colsample_bytree": np.linspace(0.5, 1.0, 6),
    "gamma": [0, 1, 5],
    "min_child_weight": np.arange(1, 10),
    "reg_alpha": [0, 0.1, 1],
    "reg_lambda": [1, 5, 10],
}

model = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42
)

search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=50,  # <-- SOLO 50 COMBINACIONES
    scoring="accuracy",
    cv=3,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

search.fit(x_train, y_train)

print(search.best_params_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
{'subsample': np.float64(0.5), 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': np.int64(300), 'min_child_weight': np.int64(3), 'max_depth': np.int64(3), 'learning_rate': np.float64(0.0027825594022071257), 'gamma': 5, 'colsample_bytree': np.float64(1.0)}


Evaluacion del modelo ajustado

In [ ]:
# Obtener el mejor modelo encontrado por RandomizedSearchCV
best_model = search.best_estimator_

# Predicciones sobre el test
y_pred = best_model.predict(x_test)

# Métricas
acc = accuracy_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1  = f1_score(y_test,  y_pred)
mcc = matthews_corrcoef(y_test, y_pred)

print("\nResultados del mejor modelo:")
print(f"Accuracy: {acc:.4f}")
print(f"Recall:   {rec:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"MCC:      {mcc:.4f}")

# Reporte completo
print("\nReporte de clasificación:\n")
print(classification_report(y_test, y_pred))


Resultados del mejor modelo:
Accuracy: 0.5735
Recall:   0.0000
F1 Score: 0.0000
MCC:      0.0000

Reporte de clasificación:

              precision    recall  f1-score   support

           0       0.57      1.00      0.73     75588
           1       0.00      0.00      0.00     56210

    accuracy                           0.57    131798
   macro avg       0.29      0.50      0.36    131798
weighted avg       0.33      0.57      0.42    131798



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# CatBoost

Librerias base

In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import accuracy_score, recall_score, f1_score, matthews_corrcoef


Modelo base

In [ ]:
train_pool = Pool(X_train, y_train)

def objective(trial):
    params = {
        "loss_function": "Logloss",
        "verbose": False,
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.1, log=True),
        "iterations": trial.suggest_int("iterations", 300, 1000),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 1, 10),
        "border_count": trial.suggest_int("border_count", 32, 128),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "grow_policy": trial.suggest_categorical(
            "grow_policy",
            ["SymmetricTree", "Depthwise", "Lossguide"]
        ),
        "max_bin": trial.suggest_int("max_bin", 128, 254),
        "scale_pos_weight": trial.suggest_int("scale_pos_weight", 1, 5),
    }

    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)

    preds = model.predict(X_val)
    return accuracy_score(y_val, preds)

# Ejecutar la optimización
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("🥇 Mejores hiperparámetros:")
print(study.best_params)

Evaluacion

In [ ]:
# Entrenar el mejor modelo
best_params = study.best_params
best_model = CatBoostClassifier(**best_params, loss_function="Logloss", verbose=False)

best_model.fit(X_train, y_train)

# Predicciones
y_pred = best_model.predict(X_test)

# Métricas
acc = accuracy_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1  = f1_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)

print("\nResultados del mejor modelo:")
print(f"Accuracy: {acc:.4f}")
print(f"Recall:   {rec:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"MCC:      {mcc:.4f}")
